In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\SisLab\\!SOYEON\\train_df.csv")
df.head()

,id,week,center_id,city_code,region_code,center_type,op_area,meal_id,category,cuisine,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,136.83,152.29,0,0,177
1,1466964,1,55,647,56,TYPE_C,2.0,1993,Beverages,Thai,136.83,135.83,0,0,270
2,1346989,1,55,647,56,TYPE_C,2.0,2539,Beverages,Thai,134.86,135.86,0,0,189
3,1338232,1,55,647,56,TYPE_C,2.0,2139,Beverages,Indian,339.50,437.53,0,0,54
4,1448490,1,55,647,56,TYPE_C,2.0,2631,Beverages,Indian,243.50,242.50,0,0,40


# onehot


In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
# center_type
data_cat = ohe.fit_transform(df[['center_type']])
df_center = pd.concat([df.drop(columns=['center_type']),pd.DataFrame(data_cat, columns=['center_type_' + str(col) for col in ohe.categories_[0]])], axis=1)

# category
data_cat = ohe.fit_transform(df[['category']])
df_category = pd.concat([df_center.drop(columns=['category']),pd.DataFrame(data_cat, columns=['category_' + str(col) for col in ohe.categories_[0]])], axis=1)

# cuisine
data_cat = ohe.fit_transform(df[['cuisine']])
train_df = pd.concat([df_category.drop(columns=['cuisine']),pd.DataFrame(data_cat, columns=['cuisine_' + str(col) for col in ohe.categories_[0]])], axis=1)

train_df.head()

,id,week,center_id,city_code,region_code,op_area,meal_id,checkout_price,base_price,emailer_for_promotion,...,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Continental,cuisine_Indian,cuisine_Italian,cuisine_Thai
0,1379560,1,55,647,56,2.0,1885,136.83,152.29,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1466964,1,55,647,56,2.0,1993,136.83,135.83,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1346989,1,55,647,56,2.0,2539,134.86,135.86,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1338232,1,55,647,56,2.0,2139,339.50,437.53,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1448490,1,55,647,56,2.0,2631,243.50,242.50,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# split

In [4]:
from sklearn.model_selection import train_test_split

X = train_df.drop('num_orders', axis=1)
y = train_df['num_orders']

In [5]:
# train, test를 8:2로 나누기
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
# shape 확인
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(365238, 32)
(91310, 32)
(365238,)
(91310,)


# scaling

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_x)
X_train_transformed = pd.DataFrame(scaler.transform(train_x))
print("\nMinMaxScaler: \n", X_train_transformed.describe())


MinMaxScaler: 
                   0              1              2              3   \
count  365238.000000  365238.000000  365238.000000  365238.000000   
mean        0.500463       0.512199       0.409727       0.566405   
std         0.288605       0.288422       0.261217       0.257554   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.250648       0.263889       0.187500       0.377432   
50%         0.500666       0.520833       0.375000       0.544747   
75%         0.750459       0.763889       0.568182       0.758755   
max         1.000000       1.000000       1.000000       1.000000   

                  4              5              6              7   \
count  365238.000000  365238.000000  365238.000000  365238.000000   
mean        0.480382       0.521916       0.508245       0.381322   
std         0.251985       0.179139       0.289063       0.177079   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.15

# Random Forest

In [8]:
# 모델 선언
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rfr = RandomForestRegressor()

# Grid

In [9]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

In [10]:
gs_s = GridSearchCV(rfr, param_grid = params,  scoring = 'neg_mean_absolute_error')
gs_s.fit(train_x, train_y)

print("Best Parameters : ", gs_s.best_params_)
print("Best Score : ", gs_s.best_score_)

Best Parameters :  {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 100}
Best Score :  -93.14880040413281


In [11]:
test_gfs = gs_s.predict(test_x)
test_gfs

array([ 55.46666517, 305.67894781,  66.73821117, ..., 191.56087954,
       305.97111182,  38.56165484])

In [12]:
mean_squared_error(test_y, test_gfs, squared = True)**0.5

187.06133165766892

In [13]:
from sklearn.metrics import r2_score
print(r2_score(test_y, test_gfs))

0.7761488944169028


# Random

In [14]:
from sklearn.model_selection import RandomizedSearchCV

In [15]:
rfg = RandomForestRegressor()

# Random Search 설정.
rs_model = RandomizedSearchCV(rfg, params)

In [16]:
rs_model.fit(train_x, train_y)

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': [6, 8, 10, 12],
                                        'min_samples_leaf': [8, 12, 18],
                                        'min_samples_split': [8, 16, 20],
                                        'n_estimators': [10, 100]})

In [17]:
test_rs = rs_model.predict(test_x)
test_rs

array([ 56.9933163 , 315.42697691,  69.32899252, ..., 185.2363508 ,
       315.42697691,  39.44973243])

In [18]:
mean_squared_error(test_y, test_rs, squared = True)**0.5

188.4688329852593

In [19]:
print(r2_score(test_y, test_rs))

0.7727675855675611
